## Combinar todos los datasets generados por usuario
En este codigo, me encargo de realizar la combinacion de los datos en grupos de 5 minutos, eso incluye los datos de las comidas, etc. Esto con el fin de obtener las 69 caracteristicas.

In [1196]:
import pandas as pd
import numpy as np

PACIENTE = '001'
PATH_FOLDER = 'F:\\Jesus\\UNSA\\Maestria\\Tesis\\Codigo Test\\PreTesisMasterRepo\Dataset\\'+PACIENTE+'\\'
PATH_FOLDER_DEMOGRAPHIC = 'G:\\Dataset\\big-ideas-lab-glycemic-variability-and-wearable-device-data-1.1.2\\'

In [1197]:
# Cargar todos los datos
# Leyendo los CSV
acc_values = pd.read_csv(PATH_FOLDER + 'ACC_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
actfis_values = pd.read_csv(PATH_FOLDER + 'actividadFisica_'+PACIENTE+'.csv', engine='python', na_values="not available")
bvp_values = pd.read_csv(PATH_FOLDER + 'BVP_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
dexcom_values = pd.read_csv(PATH_FOLDER + 'Dexcom_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
eda_values = pd.read_csv(PATH_FOLDER + 'EDA_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
hr_values = pd.read_csv(PATH_FOLDER + 'HR_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
ibi_values = pd.read_csv(PATH_FOLDER + 'IBI_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
ibi_vfc_values = pd.read_csv(PATH_FOLDER + 'IBI_5min_hr_data_'+PACIENTE+'.csv', engine='python', na_values="not available")
temp_values = pd.read_csv(PATH_FOLDER + 'TEMP_5min_'+PACIENTE+'.csv', engine='python', na_values="not available")
ritmocirc_values = pd.read_csv(PATH_FOLDER + 'ritmo_circadiano_min_'+PACIENTE+'.csv', engine='python', na_values="not available")
food_2h_values = pd.read_csv(PATH_FOLDER + 'Food_2h_'+PACIENTE+'.csv', engine='python', na_values="not available")
food_8h_values = pd.read_csv(PATH_FOLDER + 'Food_8h_'+PACIENTE+'.csv', engine='python', na_values="not available")
food_24h_values = pd.read_csv(PATH_FOLDER + 'Food_24h_'+PACIENTE+'.csv', engine='python', na_values="not available")

In [1198]:
# Leemos la data demografica tambien
demographic_values = pd.read_csv(PATH_FOLDER_DEMOGRAPHIC + 'Demographics.csv', engine='python', na_values="not available")

In [1199]:
dexcom_values.head()

,datetime,glucose_x,glucose_y,nivel
0,2020-02-13 17:20:00,61.0,101.45,PersNorm
1,2020-02-13 17:25:00,59.0,101.45,PersNorm
2,2020-02-13 17:30:00,58.0,101.45,PersNorm
3,2020-02-13 17:35:00,59.0,101.45,PersNorm
4,2020-02-13 17:40:00,63.0,101.45,PersNorm


In [1200]:
food_2h_values.head()

,datetime,calorie_sum,calorie_mean,total_carb_sum,total_carb_mean,sugar_sum,sugar_mean,protein_sum,protein_mean,cantidad_comidas_,media_comidas_
0,2020-02-13 18:00:00,456.0,456.0,85.0,85.00,83.0,83.0,16.0,16.0,1,0.570093
1,2020-02-13 20:00:00,488.0,244.0,2.5,1.25,0.8,0.4,63.4,31.7,2,0.570093
2,2020-02-13 22:00:00,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.570093
3,2020-02-14 00:00:00,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.570093
4,2020-02-14 02:00:00,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.570093


Como vemos, food tiene unos campos diferentes a los de los tiempo de los otros datasets

In [1201]:
demographic_values.head()

,ID,Gender,HbA1c
0,13,MALE,5.7
1,1,FEMALE,5.5
2,3,FEMALE,5.9
3,4,FEMALE,6.4
4,5,FEMALE,5.7


La data demografica contiene el genero y HbA1c el cual es una medida tomada en un laboratorio que tambien influye en la deteccion de la glucosa

* En este caso solo consideraremos tomar la fila de datos correspondiente al ID del paciente

In [1202]:
# Identificador único
id_to_find = int(PACIENTE)  # Convertir a entero

# Filtrar la fila correspondiente al ID
demographic_result = demographic_values.loc[demographic_values['ID'] == id_to_find]

# Mostrar el resultado
print(demographic_result)

   ID  Gender  HbA1c
1   1  FEMALE    5.5


In [1203]:
demographic_result.head()

,ID,Gender,HbA1c
1,1,FEMALE,5.5


In [1204]:
# Formateamos la data de 2, 8 y 24 horas a 5 minutos
# food_2h_values['datetime'] = pd.to_datetime(food_2h_values['time_begin'])
# food_8h_values['datetime'] = pd.to_datetime(food_8h_values['time_begin'])
# food_24h_values['datetime'] = pd.to_datetime(food_24h_values['time_begin'])

food_2h_values = food_2h_values.drop(columns=['calorie_sum'])
food_8h_values = food_8h_values.drop(columns=['calorie_sum','calorie_mean','total_carb_sum','total_carb_mean','sugar_sum','sugar_mean','protein_sum','protein_mean'])
food_24h_values = food_24h_values.drop(columns=['calorie_sum','calorie_mean','total_carb_sum','total_carb_mean','sugar_sum','sugar_mean','protein_sum','protein_mean'])

food_2h_values.head()

,datetime,calorie_mean,total_carb_sum,total_carb_mean,sugar_sum,sugar_mean,protein_sum,protein_mean,cantidad_comidas_,media_comidas_
0,2020-02-13 18:00:00,456.0,85.0,85.00,83.0,83.0,16.0,16.0,1,0.570093
1,2020-02-13 20:00:00,244.0,2.5,1.25,0.8,0.4,63.4,31.7,2,0.570093
2,2020-02-13 22:00:00,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.570093
3,2020-02-14 00:00:00,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.570093
4,2020-02-14 02:00:00,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0,0.570093


### Aqui tiene que liderar dexcom_values debido a que contiene el flag de glucosa donde indicaremos si es alto, medio o bajo en glucosa

In [1205]:
dexcom_values.head()

,datetime,glucose_x,glucose_y,nivel
0,2020-02-13 17:20:00,61.0,101.45,PersNorm
1,2020-02-13 17:25:00,59.0,101.45,PersNorm
2,2020-02-13 17:30:00,58.0,101.45,PersNorm
3,2020-02-13 17:35:00,59.0,101.45,PersNorm
4,2020-02-13 17:40:00,63.0,101.45,PersNorm


**Añadir los valores demograficos**

In [1206]:
# Realizar la combinación de los DataFrames usando pd.merge()
# Comenzamos con el primer DataFrame
dexcom_values = dexcom_values.rename(columns={'glucose_x':'glucose','glucose_y':'glucose_24h_mean'})
final_df = dexcom_values
demographic_values_row = demographic_result.iloc[0]

final_df['gender'] = demographic_values_row['Gender']
final_df['hba1c'] = demographic_values_row['HbA1c']
final_df['id'] = demographic_values_row['ID']
# Obtener los valores de la fila 1

final_df.head()

,datetime,glucose,glucose_24h_mean,nivel,gender,hba1c,id
0,2020-02-13 17:20:00,61.0,101.45,PersNorm,FEMALE,5.5,1
1,2020-02-13 17:25:00,59.0,101.45,PersNorm,FEMALE,5.5,1
2,2020-02-13 17:30:00,58.0,101.45,PersNorm,FEMALE,5.5,1
3,2020-02-13 17:35:00,59.0,101.45,PersNorm,FEMALE,5.5,1
4,2020-02-13 17:40:00,63.0,101.45,PersNorm,FEMALE,5.5,1


### Empezamos por validar que datos necesitamos para cada dataset
Para este punto vamos a evaluar cada uno de los datasets y verificar que columnas necesitamos de cada uno

**EDA**

In [1207]:
eda_values.head()

,datetime,mean,median,max,min,std,total_peaks,mean_peaks,estres,q1,q3
0,2020-02-13 15:25:00,0.038783,0.028186,0.135805,0.000000,0.027591,0,0.000000,0,0.016335,0.059254
1,2020-02-13 15:30:00,0.077960,0.079433,0.098651,0.049966,0.006985,0,0.000000,0,0.073027,0.083276
2,2020-02-13 15:35:00,0.248319,0.088401,0.885560,0.080714,0.249663,2,0.001667,1,0.085839,0.487167
3,2020-02-13 15:40:00,0.224158,0.142211,0.647261,0.031015,0.164541,0,0.000000,0,0.128118,0.314155
4,2020-02-13 15:45:00,0.107635,0.105056,0.130680,0.081995,0.008446,0,0.000000,0,0.101213,0.111462


In [1208]:
# Eliminemos las columnas que en este caso no necesitamos
eda_values = eda_values.drop(columns=['total_peaks','mean_peaks'])
# Renombrar las columnas para mayor claridad
# Agregando 'eda_' como prefijo
eda_values.rename(columns=lambda x: f'eda_{x}' if x != 'datetime' else x, inplace=True)
eda_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2125 entries, 0 to 2124
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2125 non-null   object 
 1   eda_mean    2125 non-null   float64
 2   eda_median  2125 non-null   float64
 3   eda_max     2125 non-null   float64
 4   eda_min     2125 non-null   float64
 5   eda_std     2125 non-null   float64
 6   eda_estres  2125 non-null   int64  
 7   eda_q1      2125 non-null   float64
 8   eda_q3      2125 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 149.5+ KB


**ACC**

In [1209]:
acc_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-02-13 15:25:00,64.461301,63.158531,133.779670,9.433981,9.513702,61.365707,66.558245
1,2020-02-13 15:30:00,63.274924,62.944420,140.815482,27.946377,4.404233,62.369865,63.576725
2,2020-02-13 15:35:00,63.673401,63.584589,185.652902,25.019992,4.428491,63.063460,63.820060
3,2020-02-13 15:40:00,63.676446,63.639610,167.116726,28.071338,4.135052,62.657801,64.476740
4,2020-02-13 15:45:00,64.045133,63.150614,165.692486,21.563859,9.630106,60.539243,66.618316


In [1210]:
# Renombrar las columnas para mayor claridad
# Agregando 'acc_' como prefijo
acc_values.rename(columns=lambda x: f'acc_{x}' if x != 'datetime' else x, inplace=True)
acc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2623 non-null   object 
 1   acc_mean    2125 non-null   float64
 2   acc_median  2125 non-null   float64
 3   acc_max     2125 non-null   float64
 4   acc_min     2125 non-null   float64
 5   acc_std     2125 non-null   float64
 6   acc_q1      2125 non-null   float64
 7   acc_q3      2125 non-null   float64
dtypes: float64(7), object(1)
memory usage: 164.1+ KB


**Actividad Fisica**

In [1211]:
actfis_values.head()

,datetime,mean_x,median_x,max_x,min_x,std_x,q1_x,q3_x,mean_y,median_y,max_y,min_y,std_y,q1_y,q3_y,per_act
0,2020-02-13 15:25:00,64.461301,63.158531,133.779670,9.433981,9.513702,61.365707,66.558245,85.130500,83.425,99.00,81.77,3.425507,82.880,87.5575,0
1,2020-02-13 15:30:00,63.274924,62.944420,140.815482,27.946377,4.404233,62.369865,63.576725,90.787867,95.150,101.33,71.75,9.115256,83.135,97.6225,0
2,2020-02-13 15:35:00,63.673401,63.584589,185.652902,25.019992,4.428491,63.063460,63.820060,71.166600,71.495,87.90,57.42,8.577861,62.960,77.4650,0
3,2020-02-13 15:40:00,63.676446,63.639610,167.116726,28.071338,4.135052,62.657801,64.476740,90.404500,88.140,117.25,66.32,12.884864,81.715,97.5425,1
4,2020-02-13 15:45:00,64.045133,63.150614,165.692486,21.563859,9.630106,60.539243,66.618316,74.408167,74.345,87.12,62.53,5.999222,69.665,78.5725,0


In [1212]:
actfis_values_final = actfis_values[['datetime', 'per_act']]
actfis_values_final.rename(columns=lambda x: f'actfis{x}' if x != 'datetime' else x, inplace=True)
actfis_values_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2124 entries, 0 to 2123
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   datetime       2124 non-null   object
 1   actfisper_act  2124 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 33.3+ KB


C:\Users\Alpha\AppData\Local\Temp\ipykernel_7000\687457855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actfis_values_final.rename(columns=lambda x: f'actfis{x}' if x != 'datetime' else x, inplace=True)


**BVP**

In [1213]:
bvp_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-02-13 15:25:00,0.098167,-0.200,963.62,-671.54,105.020335,-42.1500,37.6625
1,2020-02-13 15:30:00,-0.025119,0.180,251.04,-469.78,30.367606,-3.7600,3.7800
2,2020-02-13 15:35:00,0.002832,0.180,203.76,-374.16,21.528509,-1.4900,1.5100
3,2020-02-13 15:40:00,-0.000613,0.000,179.16,-214.48,24.298606,-2.8100,2.2825
4,2020-02-13 15:45:00,-0.011019,-0.015,292.87,-556.94,43.501639,-13.4425,15.0525


In [1214]:
# Renombrar las columnas para mayor claridad
# Agregando 'bvp_' como prefijo
bvp_values.rename(columns=lambda x: f'bvp_{x}' if x != 'datetime' else x, inplace=True)
bvp_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2623 non-null   object 
 1   bvp_mean    2125 non-null   float64
 2   bvp_median  2125 non-null   float64
 3   bvp_max     2125 non-null   float64
 4   bvp_min     2125 non-null   float64
 5   bvp_std     2125 non-null   float64
 6   bvp_q1      2125 non-null   float64
 7   bvp_q3      2125 non-null   float64
dtypes: float64(7), object(1)
memory usage: 164.1+ KB


**HR**

In [1215]:
hr_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-02-13 15:25:00,85.130500,83.425,99.00,81.77,3.425507,82.880,87.5575
1,2020-02-13 15:30:00,90.787867,95.150,101.33,71.75,9.115256,83.135,97.6225
2,2020-02-13 15:35:00,71.166600,71.495,87.90,57.42,8.577861,62.960,77.4650
3,2020-02-13 15:40:00,90.404500,88.140,117.25,66.32,12.884864,81.715,97.5425
4,2020-02-13 15:45:00,74.408167,74.345,87.12,62.53,5.999222,69.665,78.5725


In [1216]:
# Renombrar las columnas para mayor claridad
# Agregando 'hr_' como prefijo
hr_values.rename(columns=lambda x: f'hr_{x}' if x != 'datetime' else x, inplace=True)
hr_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   2623 non-null   object 
 1   hr_mean    2124 non-null   float64
 2   hr_median  2124 non-null   float64
 3   hr_max     2124 non-null   float64
 4   hr_min     2124 non-null   float64
 5   hr_std     2124 non-null   float64
 6   hr_q1      2124 non-null   float64
 7   hr_q3      2124 non-null   float64
dtypes: float64(7), object(1)
memory usage: 164.1+ KB


**IBI**

In [1217]:
ibi_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-02-13 15:30:00,0.903166,0.890666,0.984420,0.828163,0.059910,0.875040,0.937543
1,2020-02-13 15:35:00,0.849333,0.921917,1.140677,0.468771,0.228782,0.625028,1.039110
2,2020-02-13 15:40:00,0.930846,0.953168,1.078174,0.437520,0.159200,0.910197,1.023484
3,2020-02-13 15:45:00,0.953820,0.953169,1.250057,0.562526,0.157979,0.890666,1.046923
4,2020-02-13 15:50:00,0.937543,0.968794,1.125051,0.734409,0.098188,0.859414,1.000046


In [1218]:
# Renombrar las columnas para mayor claridad
# Agregando 'ibi_' como prefijo
ibi_values.rename(columns=lambda x: f'ibi_{x}' if x != 'datetime' else x, inplace=True)
ibi_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2621 entries, 0 to 2620
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2621 non-null   object 
 1   ibi_mean    2621 non-null   float64
 2   ibi_median  2621 non-null   float64
 3   ibi_max     2621 non-null   float64
 4   ibi_min     2621 non-null   float64
 5   ibi_std     2621 non-null   float64
 6   ibi_q1      2076 non-null   float64
 7   ibi_q3      2076 non-null   float64
dtypes: float64(7), object(1)
memory usage: 163.9+ KB


**TEMP**

In [1219]:
temp_values.head()

,datetime,mean,median,max,min,std,q1,q3
0,2020-02-13 15:25:00,29.332286,30.26,30.95,24.89,1.913087,28.410,30.79
1,2020-02-13 15:30:00,31.017333,31.01,31.11,30.87,0.044645,30.990,31.05
2,2020-02-13 15:35:00,30.833467,30.83,30.97,30.75,0.048394,30.805,30.85
3,2020-02-13 15:40:00,30.712667,30.69,30.81,30.63,0.048085,30.670,30.77
4,2020-02-13 15:45:00,30.668267,30.67,30.73,30.59,0.029400,30.650,30.69


In [1220]:
# Renombrar las columnas para mayor claridad
# Agregando 'temp_' como prefijo
temp_values.rename(columns=lambda x: f'temp_{x}' if x != 'datetime' else x, inplace=True)
temp_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   datetime     2623 non-null   object 
 1   temp_mean    2125 non-null   float64
 2   temp_median  2125 non-null   float64
 3   temp_max     2125 non-null   float64
 4   temp_min     2125 non-null   float64
 5   temp_std     2125 non-null   float64
 6   temp_q1      2125 non-null   float64
 7   temp_q3      2125 non-null   float64
dtypes: float64(7), object(1)
memory usage: 164.1+ KB


**VFC**

In [1221]:
ibi_vfc_values.head()

,datetime,ibi,MeanRR,MeanHR,MinHR,MaxHR,SDNN,RMSSD,NNx,pNNx,PowerVLF,PowerLF,PowerHF,PowerTotal,LF/HF,PeakVLF,PeakLF,PeakHF,FractionLF,FractionHF
0,2020-02-13 15:30:00,0.903166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2020-02-13 15:35:00,0.849333,816.3,75.3,60.0,101.0,124.3,241.2,22.0,73.3,2916.12,5027.74,11801.01,19744.87,0.43,0.04,0.12,0.16,29.88,70.12
2,2020-02-13 15:40:00,0.930846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2020-02-13 15:45:00,0.953820,948.5,63.3,60.7,65.8,22.3,151.3,15.0,65.2,0.00,10821.08,3557.89,14378.97,3.04,0.00,0.09,0.18,75.26,24.74
4,2020-02-13 15:50:00,0.937543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [1222]:
# Renombrar las columnas para mayor claridad
# Agregando 'vfc_' como prefijo
ibi_vfc_values.rename(columns=lambda x: f'vfc_{x}' if x != 'datetime' else x, inplace=True)
ibi_vfc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076 entries, 0 to 2075
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime        2076 non-null   object 
 1   vfc_ ibi        2076 non-null   float64
 2   vfc_MeanRR      2076 non-null   float64
 3   vfc_MeanHR      2076 non-null   float64
 4   vfc_MinHR       2076 non-null   float64
 5   vfc_MaxHR       2076 non-null   float64
 6   vfc_SDNN        2076 non-null   float64
 7   vfc_RMSSD       2076 non-null   float64
 8   vfc_NNx         2076 non-null   float64
 9   vfc_pNNx        2076 non-null   float64
 10  vfc_PowerVLF    2076 non-null   float64
 11  vfc_PowerLF     2076 non-null   float64
 12  vfc_PowerHF     2076 non-null   float64
 13  vfc_PowerTotal  2076 non-null   float64
 14  vfc_LF/HF       2065 non-null   float64
 15  vfc_PeakVLF     2076 non-null   float64
 16  vfc_PeakLF      2076 non-null   float64
 17  vfc_PeakHF      2076 non-null   f

In [1223]:
columns_to_drop_vfc = ['vfc_ ibi','vfc_MeanRR','vfc_PowerVLF','vfc_PowerLF','vfc_PowerHF','vfc_PowerTotal','vfc_LF/HF','vfc_PeakVLF','vfc_PeakLF','vfc_PeakHF','vfc_FractionLF','vfc_FractionHF']
ibi_vfc_values = ibi_vfc_values.drop(columns=columns_to_drop_vfc)
ibi_vfc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2076 entries, 0 to 2075
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    2076 non-null   object 
 1   vfc_MeanHR  2076 non-null   float64
 2   vfc_MinHR   2076 non-null   float64
 3   vfc_MaxHR   2076 non-null   float64
 4   vfc_SDNN    2076 non-null   float64
 5   vfc_RMSSD   2076 non-null   float64
 6   vfc_NNx     2076 non-null   float64
 7   vfc_pNNx    2076 non-null   float64
dtypes: float64(7), object(1)
memory usage: 129.9+ KB


**Ritmo Circadiano**

In [1224]:
ritmocirc_values.head()

,datetime,magnitude_mean,magnitude_std,hr_mean,hr_std,indicator_ritmo_circadiano,date,hr_mean_indicator,hr_std_indicator,magnitude_mean_indicator,magnitude_std_indicator,sum_indicator
0,2020-02-13 15:25:00,64.461301,9.513702,85.130500,3.425507,1,2020-02-13,0,1,0,0,1
1,2020-02-13 15:30:00,63.274924,4.404233,90.787867,9.115256,0,2020-02-13,0,1,1,1,3
2,2020-02-13 15:35:00,63.673401,4.428491,71.166600,8.577861,0,2020-02-13,1,1,1,1,4
3,2020-02-13 15:40:00,63.676446,4.135052,90.404500,12.884864,0,2020-02-13,0,0,1,1,2
4,2020-02-13 15:45:00,64.045133,9.630106,74.408167,5.999222,0,2020-02-13,1,1,1,0,3


In [1225]:
# Renombrar las columnas para mayor claridad
# Agregando 'ritcir_' como prefijo
ritmocirc_values.rename(columns=lambda x: f'ritcir_{x}' if x != 'datetime' else x, inplace=True)
ritmocirc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   datetime                           2623 non-null   object 
 1   ritcir_magnitude_mean              2623 non-null   float64
 2   ritcir_magnitude_std               2623 non-null   float64
 3   ritcir_hr_mean                     2623 non-null   float64
 4   ritcir_hr_std                      2623 non-null   float64
 5   ritcir_indicator_ritmo_circadiano  2623 non-null   int64  
 6   ritcir_date                        2623 non-null   object 
 7   ritcir_hr_mean_indicator           2623 non-null   int64  
 8   ritcir_hr_std_indicator            2623 non-null   int64  
 9   ritcir_magnitude_mean_indicator    2623 non-null   int64  
 10  ritcir_magnitude_std_indicator     2623 non-null   int64  
 11  ritcir_sum_indicator               2623 non-null   int64

In [1226]:
columns_to_drop_ritcir = ['ritcir_magnitude_mean','ritcir_magnitude_std','ritcir_hr_mean','ritcir_hr_std','ritcir_date','ritcir_sum_indicator','ritcir_magnitude_mean_indicator','ritcir_hr_mean_indicator','ritcir_hr_std_indicator','ritcir_magnitude_std_indicator']
ritmocirc_values = ritmocirc_values.drop(columns=columns_to_drop_ritcir)
ritmocirc_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2623 entries, 0 to 2622
Data columns (total 2 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   datetime                           2623 non-null   object
 1   ritcir_indicator_ritmo_circadiano  2623 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 41.1+ KB


**FOOD**

In [1227]:
food_24h_values.head()

,datetime,cantidad_comidas_,media_comidas_
0,2020-02-13,3,6.1
1,2020-02-14,8,6.1
2,2020-02-15,10,6.1
3,2020-02-16,8,6.1
4,2020-02-17,6,6.1


In [1228]:
# Renombrar las columnas para mayor claridad
# Agregando 'food2h_' como prefijo
food_2h_values.rename(columns=lambda x: f'food2h_{x}' if x != 'datetime' else x, inplace=True)

# Agregando 'food8h_' como prefijo
food_8h_values.rename(columns=lambda x: f'food8h_{x}' if x != 'datetime' else x, inplace=True)

# Agregando 'food24h_' como prefijo
food_24h_values.rename(columns=lambda x: f'food24h_{x}' if x != 'datetime' else x, inplace=True)
food_2h_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   datetime                  107 non-null    object 
 1   food2h_calorie_mean       38 non-null     float64
 2   food2h_total_carb_sum     107 non-null    float64
 3   food2h_total_carb_mean    38 non-null     float64
 4   food2h_sugar_sum          107 non-null    float64
 5   food2h_sugar_mean         38 non-null     float64
 6   food2h_protein_sum        107 non-null    float64
 7   food2h_protein_mean       38 non-null     float64
 8   food2h_cantidad_comidas_  107 non-null    int64  
 9   food2h_media_comidas_     107 non-null    float64
dtypes: float64(8), int64(1), object(1)
memory usage: 8.5+ KB


### Combinacion
Para este paso ya tenemos los dataframes listos para ser combinados

In [1229]:
# Revisamos el dataframe mas importante
final_df.head()

,datetime,glucose,glucose_24h_mean,nivel,gender,hba1c,id
0,2020-02-13 17:20:00,61.0,101.45,PersNorm,FEMALE,5.5,1
1,2020-02-13 17:25:00,59.0,101.45,PersNorm,FEMALE,5.5,1
2,2020-02-13 17:30:00,58.0,101.45,PersNorm,FEMALE,5.5,1
3,2020-02-13 17:35:00,59.0,101.45,PersNorm,FEMALE,5.5,1
4,2020-02-13 17:40:00,63.0,101.45,PersNorm,FEMALE,5.5,1


In [1230]:
food_2h_values['datetime'] = pd.to_datetime(food_2h_values['datetime'])
food_8h_values['datetime'] = pd.to_datetime(food_8h_values['datetime'])
food_24h_values['datetime'] = pd.to_datetime(food_24h_values['datetime'])
food_2h_values.set_index('datetime', inplace=True)
food_8h_values.set_index('datetime', inplace=True)
food_24h_values.set_index('datetime', inplace=True)

In [1231]:
# Resamplear los dataframes de menor frecuencia a 5 minutos
food_2h_resampled = food_2h_values.resample('5T').mean()
food_8h_resampled = food_8h_values.resample('5T').mean()
food_24h_resampled = food_24h_values.resample('5T').mean()
food_24h_resampled.head()

,food24h_cantidad_comidas_,food24h_media_comidas_
datetime,,
2020-02-13 00:00:00,3.0,6.1
2020-02-13 00:05:00,NaN,NaN
2020-02-13 00:10:00,NaN,NaN
2020-02-13 00:15:00,NaN,NaN
2020-02-13 00:20:00,NaN,NaN


In [1232]:
# Convertir to datetime todas las columnas
acc_values['datetime'] = pd.to_datetime(acc_values['datetime'])
actfis_values_final['datetime'] = pd.to_datetime(actfis_values_final['datetime'])
bvp_values['datetime'] = pd.to_datetime(bvp_values['datetime'])
final_df['datetime'] = pd.to_datetime(final_df['datetime'])
eda_values['datetime'] = pd.to_datetime(eda_values['datetime'])
hr_values['datetime'] = pd.to_datetime(hr_values['datetime'])
ibi_values['datetime'] = pd.to_datetime(ibi_values['datetime'])
ibi_vfc_values['datetime'] = pd.to_datetime(ibi_vfc_values['datetime'])
temp_values['datetime'] = pd.to_datetime(temp_values['datetime'])
ritmocirc_values['datetime'] = pd.to_datetime(ritmocirc_values['datetime'])
# Establecer la columna 'datetime' como índice
acc_values.set_index('datetime', inplace=True)
actfis_values_final.set_index('datetime', inplace=True)
bvp_values.set_index('datetime', inplace=True)
final_df.set_index('datetime', inplace=True)
eda_values.set_index('datetime', inplace=True)
hr_values.set_index('datetime', inplace=True)
ibi_values.set_index('datetime', inplace=True)
ibi_vfc_values.set_index('datetime', inplace=True)
temp_values.set_index('datetime', inplace=True)
ritmocirc_values.set_index('datetime', inplace=True)

# Llenar los valores nulos con interpolación
food_2h_resampled.interpolate(method='time', inplace=True)
food_8h_resampled.interpolate(method='time', inplace=True)
food_24h_resampled.interpolate(method='time', inplace=True)


C:\Users\Alpha\AppData\Local\Temp\ipykernel_7000\2476277127.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actfis_values_final['datetime'] = pd.to_datetime(actfis_values_final['datetime'])


In [1233]:
dataframes_to_merge = [final_df
,food_2h_resampled
,food_8h_resampled
,food_24h_resampled
,acc_values
,actfis_values_final
,bvp_values
,eda_values
,hr_values
,ibi_values
,ibi_vfc_values
,temp_values
,ritmocirc_values]

In [1234]:
food_2h_resampled.head()

,food2h_calorie_mean,food2h_total_carb_sum,food2h_total_carb_mean,food2h_sugar_sum,food2h_sugar_mean,food2h_protein_sum,food2h_protein_mean,food2h_cantidad_comidas_,food2h_media_comidas_
datetime,,,,,,,,,
2020-02-13 18:00:00,456.000000,85.0000,85.000000,83.000,83.000000,16.000,16.000000,1.000000,0.570093
2020-02-13 18:05:00,447.166667,81.5625,81.510417,79.575,79.558333,17.975,16.654167,1.041667,0.570093
2020-02-13 18:10:00,438.333333,78.1250,78.020833,76.150,76.116667,19.950,17.308333,1.083333,0.570093
2020-02-13 18:15:00,429.500000,74.6875,74.531250,72.725,72.675000,21.925,17.962500,1.125000,0.570093
2020-02-13 18:20:00,420.666667,71.2500,71.041667,69.300,69.233333,23.900,18.616667,1.166667,0.570093


In [1235]:
final_df = pd.concat(dataframes_to_merge, axis=1)

In [1236]:
final_df = final_df.fillna(0)
final_df.head()

,glucose,glucose_24h_mean,nivel,gender,hba1c,id,food2h_calorie_mean,food2h_total_carb_sum,food2h_total_carb_mean,food2h_sugar_sum,...,vfc_NNx,vfc_pNNx,temp_mean,temp_median,temp_max,temp_min,temp_std,temp_q1,temp_q3,ritcir_indicator_ritmo_circadiano
datetime,,,,,,,,,,,,,,,,,,,,,
2020-02-13 00:00:00,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-13 00:05:00,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-13 00:10:00,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-13 00:15:00,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-13 00:20:00,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1237]:
# Ahora limpiamos datos donde no tenemos valores de comida
# Definir las columnas que no se deben considerar
columns_to_exclude = ['glucose','glucose_24h_mean','nivel','gender','hba1c','id']
# Definir la función personalizada para eliminar cuando ya no se tienen registros de alimentos
def check_non_zero(row):
    # Verifica si todas las columnas excepto las especificadas son 0
    return not (row.drop(columns=columns_to_exclude) == 0).all()

In [1238]:
# Limpiamos datos donde no tenemos valores de glucosa
final_df = final_df[final_df['glucose'] != 0]

# Filtrar el DataFrame usando la función personalizada
# df_filtered = final_df[final_df.apply(check_non_zero, axis=1)]
# Se comento debido a que genera huecos de tiempo en la data final

print(final_df.count)

<bound method DataFrame.count of                      glucose  glucose_24h_mean     nivel  gender  hba1c   id  \
datetime                                                                       
2020-02-13 17:20:00     61.0        101.450000  PersNorm  FEMALE    5.5  1.0   
2020-02-13 17:25:00     59.0        101.450000  PersNorm  FEMALE    5.5  1.0   
2020-02-13 17:30:00     58.0        101.450000  PersNorm  FEMALE    5.5  1.0   
2020-02-13 17:35:00     59.0        101.450000  PersNorm  FEMALE    5.5  1.0   
2020-02-13 17:40:00     63.0        101.450000  PersNorm  FEMALE    5.5  1.0   
...                      ...               ...       ...     ...    ...  ...   
2020-02-22 17:30:00    139.0        103.986047  PersNorm  FEMALE    5.5  1.0   
2020-02-22 17:35:00    141.0        103.986047  PersNorm  FEMALE    5.5  1.0   
2020-02-22 17:40:00    143.0        103.986047  PersNorm  FEMALE    5.5  1.0   
2020-02-22 17:45:00    139.0        103.986047  PersNorm  FEMALE    5.5  1.0   
2020-02

In [1239]:
final_df.head()

,glucose,glucose_24h_mean,nivel,gender,hba1c,id,food2h_calorie_mean,food2h_total_carb_sum,food2h_total_carb_mean,food2h_sugar_sum,...,vfc_NNx,vfc_pNNx,temp_mean,temp_median,temp_max,temp_min,temp_std,temp_q1,temp_q3,ritcir_indicator_ritmo_circadiano
datetime,,,,,,,,,,,,,,,,,,,,,
2020-02-13 17:20:00,61.0,101.45,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,...,92.0,65.7,33.604500,33.61,33.65,33.50,0.027977,33.590,33.63,0.0
2020-02-13 17:25:00,59.0,101.45,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,...,144.0,57.1,33.171867,33.21,33.57,32.73,0.266618,32.925,33.43,1.0
2020-02-13 17:30:00,58.0,101.45,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,...,112.0,60.9,33.136333,33.18,33.47,32.75,0.251755,32.870,33.39,1.0
2020-02-13 17:35:00,59.0,101.45,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,...,37.0,60.7,33.244767,33.25,33.39,33.11,0.052050,33.210,33.27,0.0
2020-02-13 17:40:00,63.0,101.45,PersNorm,FEMALE,5.5,1.0,0.0,0.0,0.0,0.0,...,65.0,52.4,33.315067,33.34,33.43,33.21,0.068227,33.250,33.37,0.0


In [1240]:
final_df.to_csv(PACIENTE+'_dataset.csv')
print('Dataset combinado final generado con exito!')

Dataset combinado final generado con exito!
